[重要文献](https://cjb.ijournals.cn/html/cjbcn/2021/5/gc21051526.htm#zz)  
[可视化教程](https://escher.readthedocs.io/en/latest/escher-python.html)  
[GSMM模型搭建教程](https://cobrapy.readthedocs.io/en/latest/)

In [1]:
# 导入必要的库
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model, write_sbml_model
import pandas as pd
import matplotlib.pyplot as plt
from pubchempy import get_compounds
from rdkit import Chem
from Bio.Seq import Seq
from escher import Builder

In [2]:
# 加载iJO1366模型
model = read_sbml_model('iJO1366.xml')

print(f"Model: {model.id}")
print(f"Number of reactions: {len(model.reactions)}")
print(f"Number of metabolites: {len(model.metabolites)}")
print(f"Number of genes: {len(model.genes)}")

Model: iJO1366
Number of reactions: 2583
Number of metabolites: 1805
Number of genes: 1367


In [ ]:
# 设置目标函数为生物量生成（通常反应ID为 'BIOMASS_Ec_iJO1366_core_53p95M'）
model.objective = "BIOMASS_Ec_iJO1366_core_53p95M"

# 运行通量平衡分析（FBA）
solution = model.optimize()
print(f"默认培养基下的生长速率: {solution.objective_value}")

In [ ]:
# 查看当前培养基成分（哪些物质允许摄入）
print("默认培养基成分:", model.medium)

# 关闭葡萄糖，开启琥珀酸作为碳源
medium = model.medium
medium["EX_glc__D_e"] = 0.0    # 关闭葡萄糖摄入
medium["EX_succ_e"] = 10.0     # 开启琥珀酸摄入（设置最大摄入速率为10 mmol/gDW/h）
model.medium = medium

# 重新模拟生长
solution = model.optimize()
print(f"琥珀酸培养基下的生长速率: {solution.objective_value}")

In [ ]:
import pandas as pd
import numpy as np

# 加载数据并筛选高表达基因（假设前20%为高表达）
df = pd.read_csv('GSE288896_Count_table_Ec_PFAS.csv')
threshold = df.iloc[:, 1].quantile(0.8)
high_exp_genes = df[df.iloc[:, 1] >= threshold].index.tolist()
print(f"高表达基因数量: {len(high_exp_genes)}")

In [ ]:
df

In [ ]:
# 遍历所有反应，调整其通量范围
for reaction in model.reactions:
    # 检查反应是否关联高表达基因
    has_high_exp_gene = any(gene.id in high_exp_genes for gene in reaction.genes)
    
    # 若关联高表达基因，允许正向通量；否则限制活性
    if has_high_exp_gene:
        reaction.lower_bound = 0  # 允许正向通量（假设高表达基因促进反应）
    else:
        # 对非高表达基因关联的反应施加限制（例如限制为原范围的10%）
        if reaction.lower_bound < 0:
            reaction.lower_bound *= 0.1  # 反向通量限制
        reaction.upper_bound *= 0.1       # 正向通量限制

# 重新模拟生长
solution = model.optimize()
print(f"整合转录组后的生长速率: {solution.objective_value}")

In [ ]:
# 查看关键反应的模拟通量
key_reactions = ["EX_succ_e", "BIOMASS_Ec_iJO1366_core_53p95M", "ATPM"]
for rxn_id in key_reactions:
    flux = solution.fluxes[rxn_id]
    print(f"{rxn_id} 通量: {flux:.2f}")

In [ ]:
# 定义要敲除的基因列表（例如与琥珀酸代谢相关的基因）
target_genes = ["b0902", "b0903"]  # 示例基因ID（实际需根据模型确定）

# 敲除基因并模拟生长
with model:
    for gene_id in target_genes:
        gene = model.genes.get_by_id(gene_id)
        gene.knock_out()  # 关闭该基因关联的所有反应
    ko_solution = model.optimize()
    print(f"敲除{target_genes}后的生长速率: {ko_solution.objective_value}")

In [ ]:
# metabolome = pd.read_csv("metabolome.csv", index_col="metabolite_id")

# 假设检测到乙酸盐分泌，约束其交换反应
# if "ac_e" in metabolome.index:
model.reactions.EX_ac_e.lower_bound = 0.5  # 设置最小分泌速率为0.5 mmol/gDW/h

In [73]:
# 直接约束特定代谢物的通量（例如ATP消耗）
model.reactions.ATPM.lower_bound = 1.0  # ATP维持通量至少为1.0 mmol/gDW/h

In [109]:
def add_dynamic_bounds(model, y):
    """Use external concentrations to bound the uptake flux of glucose."""
    biomass, glucose = y  # expand the boundary species
    glucose_max_import = -10 * glucose / (5 + glucose)
    model.reactions.EX_glc__D_e.lower_bound = glucose_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, glucose = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with model:
        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            model, ['EX_succ_e', 'EX_glc__D_e'], ['max', 'max'])

    # Since the calculated fluxes are specific rates, we multiply them by the
    # biomass concentration to get the bulk exchange rates.
    fluxes = lex_constraints.values
    fluxes *= biomass

    # This implementation is **not** efficient, so I display the current
    # simulation time using a progress bar.
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes

dynamic_system.pbar = None


def infeasible_event(t, y):
    """
    Determine solution feasibility.

    Avoiding infeasible solutions is handled by solve_ivp's built-in event detection.
    This function re-solves the LP to determine whether or not the solution is feasible
    (and if not, how far it is from feasibility). When the sign of this function changes
    from -epsilon to positive, we know the solution is no longer feasible.

    """

    with model:

        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [ ]:
for i,reaction in enumerate(model.reactions):
    idx = reaction.id
    if idx == 'EX_glc__D_e' :
        print(i)

In [ ]:
ts = np.linspace(0, 15, 2)  # Desired integration resolution and interval
y0 = [0.1, 0.2]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

In [ ]:
sol

In [ ]:
ax = plt.subplot(111)
ax.plot(sol.t, sol.y.T[:, 0])
ax2 = plt.twinx(ax)
ax2.plot(sol.t, sol.y.T[:, 1], color='r')

ax.set_ylabel('Biomass', color='b')
ax2.set_ylabel('Glucose', color='r')

In [ ]:
# 通量变异性分析（FVA）
from cobra.flux_analysis import flux_variability_analysis
fva_result = flux_variability_analysis(model, reaction_list=["EX_succ_e", "ATPM"])
print(fva_result)